## Import Libraries and Data

In [ ]:
#import relevant libraries
import sys; sys.path
import pandas as pd
import numpy as np 
#import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.metrics import explained_variance_score, r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold

In [ ]:
#load in functional connectivity data + subj data file
fc = pd.read_csv('fc.csv', header=None).values
T = pd.read_csv('subj_data.csv',header=0)

#names of specific cognitive metrics you want to evaluate
cognition = ['Crystal','Fluid','Total','PicVocab','Reading','Flanker','CardSort','PicSeq','ListSort','ProcSpeed']


In [ ]:
#extract the specific cognitive metrics
crystal = T.CogCrystalComp_AgeAdj.values
fluid = T.CogFluidComp_AgeAdj.values
total = T.CogTotalComp_AgeAdj.values
picvocab = T.PicVocab_AgeAdj.values
reading = T.ReadEng_AgeAdj.values
flanker = T.Flanker_AgeAdj.values
cardsort = T.CardSort_AgeAdj.values
picseq = T.PicSeq_AgeAdj.values
listsort = T.ListSort_AgeAdj.values
procspeed = T.ProcSpeed_AgeAdj.values

#put them all into one array
cog_metric = np.transpose(np.asarray([crystal, fluid, total, picvocab, reading, flanker, cardsort, picseq, listsort, procspeed]))

## Set Model Parameters

In [ ]:
#set the # of cv loops and # of test loops and test size and number of cog variables to predict
perm = 100
cv_loops = 5
k = 3
train_size = .8
n_cog = np.size(cognition)

#set regression model type, and hyperparameter grid to search
regr = Ridge(normalize=True, max_iter=1000000)
alphas = np.linspace(250, 1000, num=4, endpoint=True, dtype=None)
paramGrid ={'alpha': alphas}

#set train data for males and females depending on what you want the input variable to be
X = fc

X_m = fc[T.Gender=='M',:]
Y_m = cog_metric[T.Gender=='M',:]

X_f = fc[T.Gender=='F',:]
Y_f = cog_metric[T.Gender=='F',:]

#set the number of features 
n_feat = X.shape[1]

In [ ]:
#create arrays to store coefficient of determination from different models
#bb indicates trained on both, tested on both (sex-independent model)
r2_bb = np.zeros([perm,n_cog])

#bm indicates trained on male, tested on male (sex-independent model)
r2_bm = np.zeros([perm,n_cog])

#bf indicates trained on male, tested on female (sex-independent model)
r2_bf = np.zeros([perm,n_cog])

#mm indicates trained on male, tested on male (male-specific model)
r2_mm = np.zeros([perm,n_cog])

#mf indicates trained on male, tested on female (male-specific model)
r2_mf = np.zeros([perm,n_cog])

#ff indicates trained on female, tested on female (female-specific model)
r2_ff = np.zeros([perm,n_cog])

#fm indicates trained on female, tested on male (female-specific model)
r2_fm = np.zeros([perm,n_cog])

#create variables to store explained variance
var_bb = np.zeros([perm,n_cog])
var_bm = np.zeros([perm,n_cog])
var_bf = np.zeros([perm,n_cog])
var_mm = np.zeros([perm,n_cog])
var_mf = np.zeros([perm,n_cog])
var_ff = np.zeros([perm,n_cog])
var_fm = np.zeros([perm,n_cog])

#create variables to store prediction accuracy
corr_bb = np.zeros([perm,n_cog])
corr_bm = np.zeros([perm,n_cog])
corr_bf = np.zeros([perm,n_cog])
corr_mm = np.zeros([perm,n_cog])
corr_mf = np.zeros([perm,n_cog])
corr_ff = np.zeros([perm,n_cog])
corr_fm = np.zeros([perm,n_cog])

#create variables to store optimised hyperparameters
opt_alpha_b = np.zeros([perm,n_cog])
opt_alpha_m = np.zeros([perm,n_cog])
opt_alpha_f = np.zeros([perm,n_cog])

#create variables to store output variables for test set
cogtest_m = np.zeros([perm,n_cog,int(np.ceil(X_m.shape[0]*(1-train_size)))])
cogtest_f = np.zeros([perm,n_cog,int(np.ceil(X_f.shape[0]*(1-train_size)))])
cogtest_b = np.zeros([perm,n_cog,cogtest_m.shape[2]+cogtest_f.shape[2]])

#create variabels to store predictions from the models
preds_mm = np.zeros([perm,n_cog,int(np.ceil(X_m.shape[0]*(1-train_size)))])
preds_mf = np.zeros([perm,n_cog,int(np.ceil(X_f.shape[0]*(1-train_size)))])
preds_ff = np.zeros([perm,n_cog,int(np.ceil(X_f.shape[0]*(1-train_size)))])
preds_fm = np.zeros([perm,n_cog,int(np.ceil(X_m.shape[0]*(1-train_size)))])
preds_bb = np.zeros([perm,n_cog,cogtest_m.shape[2]+cogtest_f.shape[2]])
preds_bm = np.zeros([perm,n_cog,int(np.ceil(X_m.shape[0]*(1-train_size)))])
preds_bf = np.zeros([perm,n_cog,int(np.ceil(X_f.shape[0]*(1-train_size)))])

#create variables to store feature importance
featimp_b = np.zeros([perm,n_feat,n_cog])
featimp_m = np.zeros([perm,n_feat,n_cog])
featimp_f = np.zeros([perm,n_feat,n_cog])

## Optimise and Run Models

In [ ]:
#iterate through permutations
for p in range(perm):
    print('Permutation %d' %(p+1))
    
    #split male and female data into train and test splits
    x_train_m, x_test_m, cog_train_m, cog_test_m = train_test_split(X_m, Y_m, test_size=1-train_size, shuffle=True, random_state=p)
    x_train_f, x_test_f, cog_train_f, cog_test_f = train_test_split(X_f, Y_f, test_size=1-train_size, shuffle=True, random_state=p)
    
    #concatenate train and test data across the sexes for sex-independent model
    x_train_b = np.concatenate((x_train_m, x_train_f), axis=0)
    x_test_b = np.concatenate((x_test_m, x_test_f), axis=0)
    
    cog_train_b = np.concatenate((cog_train_m, cog_train_f), axis=0)
    cog_test_b = np.concatenate((cog_test_m, cog_test_f), axis=0)
    
    #iterate through the cognitive metrics to predict
    for cog in range (n_cog):

        #print cognitive metrics being predicted 
        print ("Cognition: %s" % cognition[cog])
        
        #set y values for train and test sets for sex-independent and sex-specific models     
        y_train_b = cog_train_b[:,cog]
        y_train_m = cog_train_m[:,cog]
        y_train_f = cog_train_f[:,cog]
    
        y_test_b = cog_test_b[:,cog]
        y_test_m = cog_test_m[:,cog]
        y_test_f = cog_test_f[:,cog]
        
        #save test data
        cogtest_b[p,cog,:] = y_test_b
        cogtest_m[p,cog,:] = y_test_m
        cogtest_f[p,cog,:] = y_test_f

        #set variables to store nested CV scores, and best parameters from hyperparameter optimisation
        nested_scores_b = []
        best_params_b = []
        
        nested_scores_m = []
        best_params_m = []

        nested_scores_f = []
        best_params_f = []
        

        #optimise regression model using nested CV
        print('Training Models')
        
          
        for i in range(cv_loops):

            #print("Nested CV - %d/%d" % (i,cv_loops))

            #set parameters for inner and outer loops for CV
            inner_cv = KFold(n_splits=k, shuffle=True, random_state=i)
            outer_cv = KFold(n_splits=k, shuffle=True, random_state=i)
            
            #SEX-INDEPENDENT MODEL
            #define regressor with grid-search CV for inner loop
            gridSearch_b = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1,
                                      verbose=0, cv=inner_cv, scoring='r2')

            #fit regression
            gridSearch_b.fit(x_train_b, y_train_b)

            #save parameters corresponding to the best score
            best_params_b.append(list(gridSearch_b.best_params_.values()))

            #call cross_val_score for outer loop
            nested_score_b = cross_val_score(gridSearch_b, X=x_train_b, y=y_train_b, cv=outer_cv, 
                                           scoring='r2', verbose=1)

            #record nested CV scores
            nested_scores_b.append(np.median(nested_score_b))

            
            #MALE-SPECIFIC MODEL
            #define regressor with grid-search CV for inner loop
            gridSearch_m = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, 
                                      verbose=0, cv=inner_cv, scoring='r2')

            #fit regression
            gridSearch_m.fit(x_train_m, y_train_m)

            #save parameters corresponding to the best score
            best_params_m.append(list(gridSearch_m.best_params_.values()))

            #call cross_val_score for outer loop
            nested_score_m = cross_val_score(gridSearch_m, X=x_train_m, y=y_train_m, cv=outer_cv, 
                                           scoring='r2', verbose=1)

            #record nested CV scores
            nested_scores_m.append(np.median(nested_score_m))


            #FEMALE-SPECIFIC MODEL
            #define regressor with grid-search CV for inner loop
            gridSearch_f = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, 
                                      verbose=0, cv=inner_cv)

            #fit regression
            gridSearch_f.fit(x_train_f, y_train_f)

            #save parameters corresponding to the best score
            best_params_f.append(list(gridSearch_f.best_params_.values()))

            #call cross_val_score for outer loop
            nested_score_f = cross_val_score(gridSearch_f, X=x_train_f, y=y_train_f, cv=outer_cv, 
                                           scoring='r2', verbose=1)

            #record nested CV scores
            nested_scores_f.append(np.median(nested_score_f))
            
            print("%d/%d Complete" % (i+1,cv_loops))
            
            
        print('Testing Models')

        #once all CV loops are complete, fit models based on optimised hyperparameters

        #save optimised alpha values
        opt_alpha_b[p,cog] = np.median(best_params_b)
        opt_alpha_m[p,cog] = np.median(best_params_m)
        opt_alpha_f[p,cog] = np.median(best_params_f)


        #fit sex-independent model
        model_b = Ridge(alpha = opt_alpha_b[p,cog], normalize=True, max_iter=1000000)
        model_b.fit(x_train_b, y_train_b);
        
        #fit sex-specific models
        model_m = Ridge(alpha = opt_alpha_m[p,cog], normalize=True, max_iter=1000000)
        model_m.fit(x_train_m, y_train_m);

        model_f = Ridge(alpha = opt_alpha_f[p,cog], normalize=True, max_iter=1000000)
        model_f.fit(x_train_f, y_train_f);

        #evaluate sex-independent model when testing on both sexes, testing on males, and testing on females
        r2_bb[p,cog]=model_b.score(x_test_b,y_test_b)
        r2_bm[p,cog]=model_b.score(x_test_m,y_test_m)
        r2_bf[p,cog]=model_b.score(x_test_f,y_test_f)
        
        #evaluate sex-specific models when testing on males, and testing on females
        r2_mm[p,cog]=model_m.score(x_test_m,y_test_m)
        r2_mf[p,cog]=model_m.score(x_test_f,y_test_f)

        r2_fm[p,cog]=model_f.score(x_test_m,y_test_m)
        r2_ff[p,cog]=model_f.score(x_test_f,y_test_f)


        #generate predictions from sex-independent model
        preds_bb[p,cog,:] = model_b.predict(x_test_b).ravel()
        preds_bm[p,cog,:] = model_b.predict(x_test_m).ravel()
        preds_bf[p,cog,:] = model_b.predict(x_test_f).ravel()
        
        #generate predictions from male-specific model
        preds_mm[p,cog,:] = model_m.predict(x_test_m).ravel()
        preds_mf[p,cog,:] = model_m.predict(x_test_f).ravel()

        #generate predictions from female-specifc model
        preds_ff[p,cog,:] = model_f.predict(x_test_f).ravel()
        preds_fm[p,cog,:] = model_f.predict(x_test_m).ravel()

        
        #compute explained variance from sex-independent model
        var_bb[p,cog] = explained_variance_score(y_test_b, preds_bb[p,cog,:])
        var_bm[p,cog] = explained_variance_score(y_test_m, preds_bm[p,cog,:])
        var_bf[p,cog] = explained_variance_score(y_test_f, preds_bf[p,cog,:])
        
        #compute explained variance from male-specific model
        var_mm[p,cog] = explained_variance_score(y_test_m, preds_mm[p,cog,:])
        var_mf[p,cog] = explained_variance_score(y_test_f, preds_mf[p,cog,:])

        #compute explained variance from male-specific model
        var_ff[p,cog] = explained_variance_score(y_test_f, preds_ff[p,cog,:])
        var_fm[p,cog] = explained_variance_score(y_test_m, preds_fm[p,cog,:])


        #compute prediciton accuracy from sex-independent model
        corr_bb[p,cog] = np.corrcoef(y_test_b, preds_bb[p,cog,:])[1,0]
        corr_bm[p,cog] = np.corrcoef(y_test_m, preds_bm[p,cog,:])[1,0]
        corr_bf[p,cog] = np.corrcoef(y_test_f, preds_bf[p,cog,:])[1,0]
        
        #compute prediciton accuracy from male-specific model
        corr_mm[p,cog] = np.corrcoef(y_test_m, preds_mm[p,cog,:])[1,0]
        corr_mf[p,cog] = np.corrcoef(y_test_f, preds_mf[p,cog,:])[1,0]
        
        #compute prediciton accuracy from female-specific model
        corr_ff[p,cog] = np.corrcoef(y_test_f, preds_ff[p,cog,:])[1,0]
        corr_fm[p,cog] = np.corrcoef(y_test_m, preds_fm[p,cog,:])[1,0]


        #extract feature importance from all models        
        featimp_b[p,:,cog] = model_b.coef_
        featimp_m[p,:,cog] = model_m.coef_
        featimp_f[p,:,cog] = model_f.coef_

## Save Data

In [ ]:
#save r2 (coefficient of determination)
np.savetxt('r2_bm.txt', r2_bm, delimiter=',')
np.savetxt('r2_bf.txt', r2_bf, delimiter=',')
np.savetxt('r2_mm.txt', r2_mm, delimiter=',')
np.savetxt('r2_mf.txt', r2_mf, delimiter=',')
np.savetxt('r2_fm.txt', r2_fm, delimiter=',')
np.savetxt('r2_ff.txt', r2_ff, delimiter=',')


#save explained variance
np.savetxt('var_bm.txt', var_bm, delimiter=',')
np.savetxt('var_bf.txt', var_bf, delimiter=',')
np.savetxt('var_mm.txt', var_mm, delimiter=',')
np.savetxt('var_mf.txt', var_mf, delimiter=',')
np.savetxt('var_fm.txt', var_fm, delimiter=',')
np.savetxt('var_ff.txt', var_ff, delimiter=',')


#save prediction accuracy
np.savetxt('corr_bm.txt', corr_bm, delimiter=',')
np.savetxt('corr_bf.txt', corr_bf, delimiter=',')
np.savetxt('corr_mm.txt', corr_mm, delimiter=',')
np.savetxt('corr_mf.txt', corr_mf, delimiter=',')
np.savetxt('corr_fm.txt', corr_fm, delimiter=',')
np.savetxt('corr_ff.txt', corr_ff, delimiter=',')

#save optimised hyperparameters
np.savetxt('alpha_b.txt', opt_alpha_b, delimiter=',')
np.savetxt('alpha_m.txt', opt_alpha_m, delimiter=',')
np.savetxt('alpha_f.txt', opt_alpha_f, delimiter=',')



#save feature importances
np.savetxt('crystal_featimp_b.txt', featimp_b[:,:,0], delimiter=',')
np.savetxt('crystal_featimp_m.txt', featimp_m[:,:,0], delimiter=',')
np.savetxt('crystal_featimp_f.txt', featimp_f[:,:,0], delimiter=',')

np.savetxt('fluid_featimp_b.txt', featimp_b[:,:,1], delimiter=',')
np.savetxt('fluid_featimp_m.txt', featimp_m[:,:,1], delimiter=',')
np.savetxt('fluid_featimp_f.txt', featimp_f[:,:,1], delimiter=',')

np.savetxt('total_featimp_b.txt', featimp_b[:,:,2], delimiter=',')
np.savetxt('total_featimp_m.txt', featimp_m[:,:,2], delimiter=',')
np.savetxt('total_featimp_f.txt', featimp_f[:,:,2], delimiter=',')

np.savetxt('picvocab_featimp_b.txt', featimp_b[:,:,3], delimiter=',')
np.savetxt('picvocab_featimp_m.txt', featimp_m[:,:,3], delimiter=',')
np.savetxt('picvocab_featimp_f.txt', featimp_f[:,:,3], delimiter=',')

np.savetxt('reading_featimp_b.txt', featimp_b[:,:,4], delimiter=',')
np.savetxt('reading_featimp_m.txt', featimp_m[:,:,4], delimiter=',')
np.savetxt('reading_featimp_f.txt', featimp_f[:,:,4], delimiter=',')

np.savetxt('flanker_featimp_b.txt', featimp_b[:,:,5], delimiter=',')
np.savetxt('flanker_featimp_m.txt', featimp_m[:,:,5], delimiter=',')
np.savetxt('flanker_featimp_f.txt', featimp_f[:,:,5], delimiter=',')

np.savetxt('cardsort_featimp_b.txt', featimp_b[:,:,6], delimiter=',')
np.savetxt('cardsort_featimp_m.txt', featimp_m[:,:,6], delimiter=',')
np.savetxt('cardsort_featimp_f.txt', featimp_f[:,:,6], delimiter=',')

np.savetxt('picseq_featimp_b.txt', featimp_b[:,:,7], delimiter=',')
np.savetxt('picseq_featimp_m.txt', featimp_m[:,:,7], delimiter=',')
np.savetxt('picseq_featimp_f.txt', featimp_f[:,:,7], delimiter=',')

np.savetxt('listsort_featimp_b.txt', featimp_b[:,:,8], delimiter=',')
np.savetxt('listsort_featimp_m.txt', featimp_m[:,:,8], delimiter=',')
np.savetxt('listsort_featimp_f.txt', featimp_f[:,:,8], delimiter=',')

np.savetxt('procspeed_featimp_b.txt', featimp_b[:,:,9], delimiter=',')
np.savetxt('procspeed_featimp_m.txt', featimp_m[:,:,9], delimiter=',')
np.savetxt('procspeed_featimp_f.txt', featimp_f[:,:,9], delimiter=',')